In [293]:
import pandas as pd
import numpy as np
import datetime as datetime
import math
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
import matplotlib as plt

In [257]:
df_ts = pd.read_csv("trafficSource_fullyear2.csv")
df_ts

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Clean up and create day of week and hour columns

In [259]:
df_ts = df_ts.drop(["Unnamed: 0", "Unnamed: 0.1"], axis = 1)
df_ts['dow'] = df_ts.visitStartTime.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').weekday())
df_ts['hour'] = df_ts.visitStartTime.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').hour)
df_ts

Split data and target - transactions are traeted as yes/no - number of transactions not used. 

In [263]:
x = df_ts[["visitNumber", 'hits', 'pageviews', 'timeOnSite', 'source', 'medium', 'keyword', 'adContent',
       'page', 'slot', 'adNetworkType', 'browser', 'operatingSystem',
       'isMobile', 'deviceCategory', 'continent', 'subContinent', 'country',
       'region', 'metro', 'city', 'networkDomain', 'dow', 'hour']]
          
y = df_ts.transactions
y = np.where(y.isna(), 0, 1)
y

array([0, 0, 0, ..., 0, 0, 0])

In [264]:
for column in df_ts:
    print(df_ts[column].value_counts())

1493146175    8
1484649802    6
1481369525    6
1478345904    6
1500856602    5
             ..
1474453216    1
1495500507    1
1470621369    1
1470361302    1
1497372675    1
Name: visitId, Length: 886303, dtype: int64
1957458976293878100    257
824839726118485274     237
3608475193341679870    192
1856749147915772585    184
3269834865385146569    144
                      ... 
3233838148966045866      1
3744702469572810920      1
3921169493649938599      1
3357647141530654884      1
6652737354596300449      1
Name: fullVisitorId, Length: 731061, dtype: int64
1      703060
2       92548
3       35843
4       19157
5       11615
        ...  
306         1
349         1
350         1
351         1
395         1
Name: visitNumber, Length: 384, dtype: int64
20161128    4807
20161115    4685
20161114    4466
20161130    4435
20161026    4375
            ... 
20170514    1290
20170513    1251
20161230    1232
20161224    1231
20161231    1211
Name: date, Length: 366, dtype: int64
2017-04-2

Top    20956
RHS      504
Name: slot, dtype: int64
Cj0KEQjwmIrJBRCRmJ_x7KDo-9oBEiQAuUPKMufMpuG3ZdwYO8GTsjiBFd5MPHStZa9y_9NCrI8X97oaAglc8P8HAQ     70
Cj0KEQjw1ee_BRD3hK6x993YzeoBEiQA5RH_BEA562M9tvl_mtnAFvtDnDqOQRp1RvxMMgwjcX1LAfwaAj4o8P8HAQ     41
CJH1vbf94M8CFUElgQodyakHgQ                                                                     29
Cj0KEQiAw_DEBRChnYiQ_562gsEBEiQA4LcssmB_RWgvpPnltzlzj5rGwqx5lk87wC5CjfcqzneNZewaAiAp8P8HAQ     27
CjwKEAiAj7TCBRCp2Z22ue-zrj4SJACG7SBEJui6ggr6ocA-eDC2-lX7W1m5IA1c_qNbzwZVTqUanxoCb5rw_wcB       24
                                                                                               ..
CjwKEAjw1qHABRDU9qaXs4rtiS0SJADNzJisxkMJD5OUcR_wfG-UnJduVOeKSOFya5NaM7W4wgWQ-RoCWfPw_wcB        1
CPrXhKWFr9QCFUZAhgodL4wBJw                                                                      1
CjwKCAjw7dfKBRAdEiwAch0egfthvWECo9r9v0hJy6nnjd9w1YdMbnNleEqgKL9Tm9OluQIY3S3DJhoCk1UQAvD_BwE     1
CLTTm5O849ACFZNffgody-gEug                                         

find columns with nas and clean up

In [265]:
nacol = pd.DataFrame(x.isna().any()).reset_index()
nacol.columns = ['column', 'Bool']
nacol = nacol[nacol.Bool == True]
nacol

,column,Bool
2,pageviews,True
3,timeOnSite,True
6,keyword,True
7,adContent,True
8,page,True
9,slot,True
10,adNetworkType,True


In [266]:
for i in nacol.column:
    try:
        x[i] = np.where(np.isnan(x[i]),0, x[i])
    except:
        x[i] = np.where(x[i].isna(),"No", x[i])
#x.isna().any()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


visitNumber        False
hits               False
pageviews          False
timeOnSite         False
source             False
medium             False
keyword            False
adContent          False
page               False
slot               False
adNetworkType      False
browser            False
operatingSystem    False
isMobile           False
deviceCategory     False
continent          False
subContinent       False
country            False
region             False
metro              False
city               False
networkDomain      False
dow                False
hour               False
dtype: bool

List of categorical columns

In [267]:
xcat = ['source', 'medium', 'keyword', 'adContent', 'slot', 'adNetworkType', 'browser', 'operatingSystem',
       'isMobile', 'deviceCategory', 'continent', 'subContinent', 'country',
       'region', 'metro', 'city', 'networkDomain']

xcat

Label encode categorical columns

In [269]:
le = preprocessing.LabelEncoder()

for i in xcat:
    x[i] = le.fit_transform(x[i])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Random Forest Classifier

In [271]:
rf = RandomForestClassifier()
rf.fit(x, y)  

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [282]:
scores = cross_val_score(rf, x, y, cv=5)
scores

array([0.98709692, 0.98737903, 0.98729043, 0.9872351 , 0.98751729])

In [273]:
ypred = rf.predict(x)
confusion_matrix(y, ypred)

array([[892071,     30],
       [  1095,  10457]])

In [284]:
pd.Series(rf.feature_importances_, index=x.columns).sort_values(ascending=False)

pageviews          0.197253
timeOnSite         0.181686
hits               0.152871
hour               0.091253
dow                0.062157
visitNumber        0.058238
networkDomain      0.051916
city               0.037506
operatingSystem    0.029833
metro              0.026006
region             0.020942
country            0.020430
browser            0.011804
source             0.010985
deviceCategory     0.009755
medium             0.007374
subContinent       0.006917
keyword            0.006127
continent          0.004679
isMobile           0.004060
page               0.002274
slot               0.002261
adNetworkType      0.002045
adContent          0.001627
dtype: float64